# Układy równań liniowych

### Pojęcia warte poznania

* Układ równań liniowych: https://pl.wikipedia.org/wiki/Układ_równań_liniowych
* Rząd macierzy: https://pl.wikipedia.org/wiki/Rząd_macierzy
* Kombinacja liniowa: https://pl.wikipedia.org/wiki/Kombinacja_liniowa
* Eliminacja Gaussa: https://pl.wikipedia.org/wiki/Metoda_eliminacji_Gaussa, Kincaid-Cheney_*_ str. 245, pełny pseudokod: str. 252
* Pivoting: https://en.wikipedia.org/wiki/Pivot_element#Partial_and_complete_pivoting, K.C. str. 261, pełny pseudokod: str. 267
* Norma wektora: https://pl.wikipedia.org/wiki/Przestrze%C5%84_unormowana, K.C. str. 320
* Norma macierzy: https://pl.wikipedia.org/wiki/Norma_macierzowa
* Macierz dodatnio określona: https://pl.wikipedia.org/wiki/Określoność_formy
* Faktoryzacja LU: https://pl.wikipedia.org/wiki/Metoda_LU, K.C. str. 294
* Faktoryzacja Cholesky'ego: https://pl.wikipedia.org/wiki/Rozkład_Choleskiego, K.C. str. 305

Dodatkowo:
* Wskaźnik uwarunkowania: https://pl.wikipedia.org/wiki/Wska%C5%BAnik_uwarunkowania, K.C. str.321
* Metoda Jacobiego: https://en.wikipedia.org/wiki/Jacobi_method, K.C. 323

Książka dla wytrwałych (naprawdę): Y. Saad "Iterative Methods for Sparse Linear Systems"

In [2]:
from typing import Optional, Tuple
import numpy as np

### Zadanie rozgrzewkowe:
Napisać mnożenie macierzy w ulubionym_\**_ języku programowania.

**Pytanko:** jakie muszą być wymiary mnożonych macierzy? (Który wymiar musi się "zgadzać"?)

**Zadanko:** Uzupełnić brakujące wymiary macierzy w docstringu (z dokładnością do ["alfa-konwersji"](https://pl.wikipedia.org/wiki/Konwersja_α))

In [18]:
def agh_superfast_matrix_multiply(a: np.matrix, b: np.matrix) -> np.matrix:
    """Perform totally ordinary multiplication of matrices.
    
    :param a: matrix with dimensions 2 by 4
    :param b: matrix with dimensions 4 by 2
    :return:  matrix with dimensions 4 by 4
    """
    
    while(np.size(a,0) > np.size(b,1)):
        b = np.insert(b, np.size(b,1), values=0, axis=1)
        
    while(np.size(a,0) < np.size(b,1)):
        a = np.insert(a, np.size(a,0), values=0, axis=1)
        
    while(np.size(a,1) > np.size(b,0)):
        b = np.insert(b, np.size(b,0), values=0, axis=0)
        
    while(np.size(a,1) < np.size(b,0)):
        a = np.insert(a, np.size(a,1), values=0, axis=0)
        
    new_matrix = np.zeros(shape=(np.size(a,0),np.size(b,1)))
    
    for i in range(0,np.size(a,0)):
        for j in range(0,np.size(a,0)):
            for k in range(0,np.size(b,0)):
                new_matrix[i,j]+=a[i,k]*b[k,j]
            
    return new_matrix


m1 = np.matrix([[1, 2],
                [3, 4],
                [4, 5],
                [5, 1]])

m2 = np.matrix([[1, 2, 3],
                [4, 5, 6]])

m2 = np.insert(m2, np.size(m2,1), values=0, axis=1) 

res = agh_superfast_matrix_multiply(m1, m2)
assert np.allclose(res, m1 * m2), "Wrong multiplication result"

### Zadania
1. Przeczytać rozdz. 7. Kincaida i Cheney'a (Systems of Linear Equations).
2. Przeczytać rozdz. 8. Kincaida i Cheney'a (Additional Topics Concerning Systems of Linear Equations).
3. Napisać kod (w ulubionym_\**_ języku) do eliminacji Gaussa z i bez pivotingu.
4. Rozwiązać poniższy układ równań z pivotingiem i bez, porównać wyniki:

In [4]:
def Gauss_without_pivoting(A: np.matrix, b: np.matrix):
    for k in range(0,np.size(A,1)):
        for i in range(k+1,np.size(A,0)):
            dividor = A[i,k]/A[k,k]
            b[i] = b[i] - (dividor)*b[k]
            for j in range(k,np.size(A,1)):
                A[i,j] = A[i,j] -(dividor)*A[k,j]
    result = np.zeros(shape=(np.size(b,0),1))
    for i in reversed(range(0,np.size(result,0))):
        for j in reversed(range(i,np.size(result,0))):
            b[i] = b[i] - A[i,j]*result[j]
        result[i] = b[i]/A[i,i]
    return result


A = np.matrix([[0.0001, -5.0300, 5.8090, 7.8320],
               [2.2660, 1.9950,  1.2120, 8.0080],
               [8.8500, 5.6810,  4.5520, 1.3020],
               [6.7750, -2.253,  2.9080, 3.9700]])

b = np.matrix([9.5740, 7.2190, 5.7300, 6.2910]).transpose()

x = np.linalg.solve(A, b)
print(Gauss_without_pivoting(A,b))
print(x)

[[ 0.21602477]
 [-0.00791511]
 [ 0.63524333]
 [ 0.74617428]]
[[ 0.21602477]
 [-0.00791511]
 [ 0.63524333]
 [ 0.74617428]]


In [5]:
def Gauss_with_pivoting(A: np.matrix, b: np.matrix):
    last_sequence_of_rows = np.zeros(shape=(np.size(b,0),1))
    index_vector = np.zeros(shape=(np.size(b,0)+1,1))
    for i in range(0,np.size(index_vector,0)):
        index_vector[i] = i-1
    biggests_values_vector = np.zeros(shape=(np.size(b,0),1))
    for tmp in index_vector:
        i = (int)(tmp)
        for j in range(0,np.size(A,1)):
            if biggests_values_vector[i] < np.absolute(A[i,j]):
                biggests_values_vector[i] = np.absolute(A[i,j])
    for k in range(0,np.size(b,0)):
        index_vector = index_vector[1:np.size(index_vector,0)]
        scale_vector = np.zeros(shape=(np.size(b,0)-k,1))
        for i in range(0,np.size(index_vector,0)):
            tmp = (int)(index_vector[i])
            scale_vector[i] = np.absolute(A[tmp,k])/biggests_values_vector[tmp]
        first = index_vector.item(0)
        iterator = 0
        while(True):
            if scale_vector[iterator] == np.max(scale_vector):
                index_vector[0] = index_vector[iterator]
                index_vector[iterator] = first
                break
            iterator += 1
        last_sequence_of_rows[k] = index_vector.item(0)
        k2 = (int) (index_vector.item(0))
        for tmp in range(1,np.size(index_vector,0)):
            i2 = (int) (index_vector.item(tmp))
            dividor = A[i2,k]/A[k2,k]
            b[i2] = b[i2] - (dividor)*b[k2]
            for j2 in range(0,np.size(A,1)):
                A[i2,j2] = A[i2,j2] -(dividor)*A[k2,j2]

    result = np.zeros(shape=(np.size(b,0),1))
    for id_of_row,row in enumerate(reversed(last_sequence_of_rows)):
        int_row = (int)(row)
        for i in reversed(range(np.size(result,0) - id_of_row,np.size(result,0) )):
            elder_row = (int)(last_sequence_of_rows[i])
            b[int_row] = b[int_row] - A[int_row,i]*result[elder_row]
        result[int_row] = b[int_row]/A[int_row,np.size(result,0) - id_of_row - 1]
    last_result = np.zeros(shape=(np.size(result,0),1))
    iterator = 0
    for i in last_sequence_of_rows:
        last_result[iterator] = result[(int)(i)]
        iterator+=1
    return last_result


A = np.matrix([[3.0, -13.0, 9.0, 3.0],
               [-6.0, 4.0,  1.0, -18.0],
               [6.0, -2.0,  2.0, 4.0],
               [12.0, -8.0,  6.0, 10.0]])

b = np.matrix([-19.0, -34.0, 16.0, 26.0]).transpose()

Gauss_with_pivoting(A,b)


array([[ 3.],
       [ 1.],
       [-2.],
       [ 1.]])

**Pytanie**: dlaczego wołamy `transpose()` na wektorze `b`?
Bo wektor b musi byc "pionowy"

Sprawdźmy, czy rozwiązanie jest ok (**Pytanie'**: dlaczego po prostu nie użyjemy `==` lub jakiegoś `equals`?):

In [6]:
np.allclose(np.dot(A, x), b)

False

### Zadania, c.d.

5. Zaimplementować algorytm faktoryzacji LU macierzy
6. (*) Zaimplementować funkcję sprawdzającą, czy dana macierz jest symetryczna i dodatnio określona
7. Zaimplementować algorytm faktoryzacji Cholesky'ego macierzy

In [43]:
def agh_superfast_lu(a: np.matrix) -> Optional[Tuple[np.matrix, np.matrix]]:
    """Perform LU decomposition of a matrix.
    
    :param a: _
    :return:  _
    """
    u = np.zeros(shape=(np.size(a,0),np.size(a,1)))
    l = np.zeros(shape=(np.size(a,0),np.size(a,1)))
    for i in range(0,np.size(a,0)):
        l[i,i] = 1
        for j in range(0,i+1):
            sum1 = 0
            for k in range(0,j):
                sum1 += l[j,k]*u[k,i]
            u[j,i] = a[j,i] - sum1

        for j in range(i+1,np.size(a,0)):
            sum2 = 0
            for k in range(0,i):
                sum2 += l[j,k]*u[k,i]
            sum2 = a[j,i] - sum2
            l[j,i] = sum2/u[i,i]

    return(u,l)

def agh_superfast_check_spd(a: np.matrix) -> bool:
    """Check whether a matrix is symmetric and positive-definite (SPD).
    
    :param a: normal matrix
    """
    l = agh_superfast_cholesky(a)
    if(np.array_equal(agh_superfast_matrix_multiply(l, l.transpose()),a)):
        return true
    return false

def agh_superfast_cholesky(a: np.matrix):
    """Perform a Cholesky decomposition of a matrix.
    
    :param a: matrix which is symetric and positive-definite
    :return:  l
    """
    l = np.zeros(shape=(np.size(a,0),np.size(a,1)))
    for k in range(0,np.size(a,0)):
        sum1 = 0
        for s in range(0,k):
            sum1+=l[k,s]*l[k,s]
        l[k,k] = np.sqrt(a[k,k] - sum1)
        
        for i in range(k+1,np.size(a,0)):
            sum2 = 0
            for s in range(0,k):
                sum2 += l[i,s]*l[k,s]
            l[i,k] = (a[i,k] - sum2)/l[k,k]    
    if(np.array_equal(agh_superfast_matrix_multiply(l, l.transpose()),a)):
        return l
    return None

A = np.matrix([[25.0, 15.0, -5.0],
               [15.0, 18.0,  0.0],
               [-5.0, 0.0,  11.0]])

A2 = np.matrix([[4.0, 12.0, -16.0],
               [12.0, 37.0,  -43.0],
               [-16.0, -43.0,  98.0]])

# agh_superfast_lu(A)
# agh_superfast_cholesky(A)

array([[ 5.,  0.,  0.],
       [ 3.,  3.,  0.],
       [-1.,  1.,  3.]])

### Zadania, opcjonalnie
5. zaimplementować metodę Jacobiego (iteracyjne rozwiązywanie układu równań liniowych)
6. za pomocą tejże metody rozwiązać powyższy układ równań

\*  wszystkie referencje odnoszą się do [książki](https://wiki.iiet.pl/lib/exe/fetch.php?media=studia:przedmioty:mownit:numerical_mathematics_and_computing.pdf) David Kincaid, Ward Cheney - "Numerical Mathematics and Computing, 6th edition"
\** _ulubiony_ język programowania staramy się pojmować rozsądnie, tj. z wyłączeniem języków pokroju Prologa, języków z [tej listy](https://en.wikipedia.org/wiki/Esoteric_programming_language) oraz Assemblera i PHP. Haskella można używać na własną odpowiedzialność.